# Partie 4: Apprentissage supervisé (regression)


Dans le cadre de notre étude, nous avons étendu l’analyse du cancer du sein à une approche de régression, dont l’objectif n’est plus de prédire la nature bénigne ou maligne d’une masse, mais d’estimer une caractéristique morphologique continue directement liée à la structure de la tumeur. Parmi l’ensemble des mesures disponibles, la variable **area1**, correspondant à l’aire moyenne de la cellule ou de la masse observée, a été retenue comme cible pour la régression. 


Ce choix s’appuie sur plusieurs arguments :

(1) l’aire constitue l’un des indicateurs les plus directs de la taille et du volume tumoral, des facteurs cliniquement associés à la gravité potentielle ;

(2) il s’agit d’une variable continue, suffisamment bien distribuée pour permettre une modélisation robuste ;

(3) elle présente une  corrélation avec des variables clés telles que le rayon, le périmètre ou la compacité, ce qui facilite la création de modèles prédictifs performants. 

En choisissant area1 comme cible, nous visons donc à construire un modèle de régression capable d’estimer la taille moyenne d'une lésion à partir des autres caractéristiques morphologiques, ce qui constitue un outil pertinent pour l’évaluation préliminaire du risque clinique.

## I. Etude de la distribution de la variable cible area1

In [ ]:
df, train, test, X_train, y_train, X_test, y_test = chargement_donnees_ucirepo(target='area1')

En se referant aux études réalisées dans la partie , on a remarqué la variable area1 présente beaucoup de valeurs abérantes, donc nous utiliserons une transformation logarithmique afin de réduire l’échelle, atténuer les outliers et rendre les relations plus linéaires pour la régression.

In [ ]:
import numpy as np
X = df.drop(columns=['area1', 'Diagnosis'])
y = df['area1']

y_log = np.log1p(y)  # Transformation logarithmique

In [ ]:
X.isna().sum()

In [ ]:
y.describe()

In [ ]:
import matplotlib.pyplot as plt
y.plot(kind='box')
plt.title("Distribution de la variable cible 'area1'")
plt.show()

on observe :

min = 143

max = 2501

std = 352

distribution très étalée → forte variance

valeurs très différentes d’un patient à l’autre
 donc une erreur absolue (MAE) pénaliserait peu les grosses erreurs.


la meilleure métrique pour cette cible est RMSE

Parce que :

* Grande plage de valeurs*

Les prédictions éloignées de la vraie valeur doivent être plus pénalisées, ce que fait RMSE.

* Sensible aux grandes erreurs*

Si le modèle se trompe beaucoup pour des tumeurs très grandes, RMSE augmente fortement → ce qui est important pour une application médicale.

* Cohérent avec une variable continue très dispersée*

MAE donne la même importance à petites et grandes erreurs, ce qui n’est pas idéal ici.

In [ ]:
y.hist(bins=30)


In [ ]:
y_log.hist(bins=30)


## II. Modélisation

Nous allons utiliser plusieurs algorithmes de régression afin de prédire la taille moyenne des zones cellulaires (area1) à partir des autres caractéristiques morphologiques des cellules.



**Linear Regression**

Utilisé comme modèle de base simple et interprétable, permettant de vérifier si la relation entre les caractéristiques cellulaires et area1 est principalement linéaire.


**Ridge Regression**

Choisi pour réduire la sensibilité aux variables corrélées entre elles grâce à la régularisation L2, ce qui stabilise les coefficients et améliore la robustesse.


**Lasso Regression**

Retenu car il réalise en plus une sélection automatique des variables via régularisation L1, utile pour simplifier le modèle si certaines variables influencent peu area1.


**Decision Tree Regressor**

Employé car il capture naturellement les relations non linéaires entre mesures cellulaires et area1, et il offre une structure explicable.


**Random Forest Regressor**

Sélectionné pour sa capacité à réduire l’overfitting en combinant plusieurs arbres et à modéliser des interactions complexes entre caractéristiques.


**Gradient Boosting Regressor**

Utilisé pour sa capacité à corriger progressivement les erreurs du modèle précédent et à détecter des patterns subtils influençant area1.


**Bagging Regressor**

Retenu car il améliore la stabilité et limite la variance des arbres de décision, surtout lorsque les données présentent des variations locales.


**AdaBoost Regressor**

Choisi pour son approche itérative qui met plus de poids sur les prédictions difficiles, ce qui peut améliorer la précision sur les mesures atypiques.


**XGBoost Regressor**

Employé pour ses performances élevées et ses mécanismes avancés de régularisation, particulièrement adaptés aux données tabulaires du WDBC.


**LightGBM Regressor**

Sélectionné pour sa rapidité d’entraînement et sa capacité à traiter efficacement les relations non linéaires dans des datasets numériques.


**CatBoost Regressor**

Privilégié pour sa stabilité, son faible besoin de réglages et sa gestion naturelle des relations complexes sans prétraitement lourd.

In [ ]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.cluster import KMeans

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import (
    RandomForestRegressor, GradientBoostingRegressor,
    BaggingRegressor, AdaBoostRegressor
)
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import scikit_posthocs as sp

def modelisation_regression_area1_visualisee(X, y, cv_splits=5, log_activation=False):
    """
    Compare plusieurs modèles de régression pour prédire 'area1' (RMSE).
    - Ajoute une colonne 'cluster' issue d'un KMeans(2)
    - Effectue une validation croisée manuelle (KFold)
    - Calcule RMSE train/test, std, temps d'entraînement/prédiction
    - Affiche 3 graphiques : RMSE train vs test (barres annotées), boxplot (variabilité),
      temps d'entraînement vs prédiction (barres annotées)
    - Calcule un score global (à maximiser) et renvoie le DataFrame des résultats et le meilleur modèle
    """

    # copie des données pour sécurité
    X = X.copy()
    
    if log_activation:
        y_log = np.log1p(y)
        y = y_log.copy().reset_index(drop=True)

    y = y.copy().reset_index(drop=True)
    X = X.reset_index(drop=True)

    # ---- 1) Ajouter cluster (KMeans 2) ----
    kmeans = KMeans(n_clusters=2, random_state=42)
    try:
        X["cluster"] = kmeans.fit_predict(X)
    except Exception:
        # si X contient des colonnes non numériques (peu probable ici), on essaie de sélectionner numériques
        numeric_cols = X.select_dtypes(include=np.number).columns.tolist()
        X["cluster"] = kmeans.fit_predict(X[numeric_cols])

    # ---- 2) Prétraitement (StandardScaler sur colonnes numériques) ----
    numeric_features = X.select_dtypes(include=np.number).columns.tolist()
    preprocessor = ColumnTransformer(
        [('num', StandardScaler(), numeric_features)],
        remainder='passthrough'
    )

    # ---- 3) Modèles de régression ----
    models = {
        "Linear Regression": LinearRegression(),
        "KNN": KNeighborsRegressor(n_neighbors=5),
        "Decision Tree": DecisionTreeRegressor(random_state=42),
        "Random Forest": RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1),
        "Gradient Boosting": GradientBoostingRegressor(n_estimators=200, learning_rate=0.1, random_state=42),
        "Bagging": BaggingRegressor(n_estimators=50, random_state=42),
        "AdaBoost": AdaBoostRegressor(n_estimators=200, learning_rate=0.1, random_state=42),
        "XGBoost": XGBRegressor(n_estimators=200, learning_rate=0.1, max_depth=5, random_state=42, verbosity=0),
        "LightGBM": LGBMRegressor(n_estimators=200, learning_rate=0.1, random_state=42),
        "CatBoost": CatBoostRegressor(iterations=200, learning_rate=0.1, depth=6, verbose=0, random_seed=42)
    }

    # ---- 4) Structures de stockage ----
    rmse_train_means = {}
    rmse_test_means = {}
    rmse_test_stds = {}
    train_times = {}
    pred_times = {}
    boxplot_values = {}

    kf = KFold(n_splits=cv_splits, shuffle=True, random_state=42)
    print(f"\n  Début de la validation croisée (CV={cv_splits})  \n")

    # ---- 5) Boucle sur modèles, CV manuelle ----
    for name, model in models.items():
        print(f"--- Modèle : {name} ---")
        pipeline = Pipeline([("preprocess", preprocessor), ("model", model)])

        rmse_train_folds = []
        rmse_test_folds = []
        times_train_folds = []
        times_pred_folds = []

        for fold_idx, (train_idx, test_idx) in enumerate(kf.split(X), 1):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

            # entraînement
            t0 = time.time()
            try:
                pipeline.fit(X_train, y_train)
            except Exception as e:
                print(f"  Erreur fit sur fold {fold_idx} pour {name} : {e}")
                # si échec, on ajoute NaN et continue
                rmse_train_folds.append(np.nan)
                rmse_test_folds.append(np.nan)
                times_train_folds.append(np.nan)
                times_pred_folds.append(np.nan)
                continue
            t1 = time.time()

            # prédiction
            t2 = time.time()
            y_pred_train = pipeline.predict(X_train)
            y_pred_test = pipeline.predict(X_test)
            t3 = time.time()

            # RMSE
            rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
            rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))

            rmse_train_folds.append(rmse_train)
            rmse_test_folds.append(rmse_test)
            times_train_folds.append(t1 - t0)
            times_pred_folds.append(t3 - t2)

            print(f"  Fold {fold_idx} — RMSE_train: {rmse_train:.4f}, RMSE_test: {rmse_test:.4f}, "
                  f"t_train: {t1-t0:.4f}s, t_pred: {t3-t2:.4f}s")

        # moyennes et std
        rmse_train_means[name] = np.nanmean(rmse_train_folds)
        rmse_test_means[name] = np.nanmean(rmse_test_folds)
        rmse_test_stds[name] = np.nanstd(rmse_test_folds)
        train_times[name] = np.nanmean(times_train_folds)
        pred_times[name] = np.nanmean(times_pred_folds)
        boxplot_values[name] = rmse_test_folds

        print(f"→ {name} Résumé : RMSE_train_mean = {rmse_train_means[name]:.4f}, "
              f"RMSE_test_mean = {rmse_test_means[name]:.4f}, RMSE_test_std = {rmse_test_stds[name]:.4f}\n")

    print("  Fin de la CV  \n")

    # ---- 6) DataFrame récapitulatif ----
    results_df = pd.DataFrame({
        "RMSE_train_mean": rmse_train_means,
        "RMSE_test_mean": rmse_test_means,
        "RMSE_test_std": rmse_test_stds,
        "Train_time": train_times,
        "Pred_time": pred_times
    })

    # ---- 7) Score global (à maximiser). On minimise RMSE, donc on prend négatif.
    alpha = 0.1   # pénalisation pour la variance
    beta = 0.01   # pénalisation pour le temps de prédiction
    results_df["Score_global"] = - (results_df["RMSE_test_mean"]
                                    + alpha * results_df["RMSE_test_std"]
                                    + beta * results_df["Pred_time"])

    best_model_name = results_df["Score_global"].idxmax()
    print("  Meilleur modèle identifié  ")
    print(results_df.loc[best_model_name])
    print("\n")

    # ---- 8) Fonctions d'annotation ----
    def annotate_bars(ax):
        for bar in ax.patches:
            h = bar.get_height()
            ax.annotate(f"{h:.3f}",
                        xy=(bar.get_x() + bar.get_width() / 2, h),
                        xytext=(0, 5),
                        textcoords="offset points",
                        ha="center", va="bottom", fontsize=9)
            

    

    # ---- 9) Graphiques ----
    models_list = list(rmse_train_means.keys())
    x = np.arange(len(models_list))
    width = 0.35

    # Graph 1 : RMSE train vs test
    fig, ax = plt.subplots(figsize=(12, 6))
    bars1 = ax.bar(x - width/2, [results_df.loc[m, "RMSE_train_mean"] for m in models_list], width, label="RMSE Train")
    bars2 = ax.bar(x + width/2, [results_df.loc[m, "RMSE_test_mean"] for m in models_list], width, label="RMSE Test")
    ax.set_xticks(x)
    ax.set_xticklabels(models_list, rotation=45, ha='right')
    ax.set_ylabel("RMSE")
    ax.set_title("RMSE moyen (Train vs Test)")
    ax.legend()
    annotate_bars(ax)
    plt.tight_layout()
    plt.show()

    # Graph 2 : Robustesse (boxplot RMSE test par fold)
    plt.figure(figsize=(12, 6))
    df_box = pd.DataFrame(boxplot_values)
    sns.boxplot(data=df_box)
    plt.title("Robustesse des modèles (variabilité RMSE-test sur les folds)")
    plt.ylabel("RMSE (test)")
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

    # Graph 3 : Temps entraînement vs prédiction
    fig, ax = plt.subplots(figsize=(12, 6))
    bars3 = ax.bar(x - width/2, [results_df.loc[m, "Train_time"] for m in models_list], width, label="Train time (s)")
    bars4 = ax.bar(x + width/2, [results_df.loc[m, "Pred_time"] for m in models_list], width, label="Pred time (s)")
    ax.set_xticks(x)
    ax.set_xticklabels(models_list, rotation=45, ha='right')
    ax.set_title("Temps moyen d'entraînement et de prédiction")
    ax.legend()
    annotate_bars(ax)
    plt.tight_layout()
    plt.show()


    # ---- 10) Test de Friedman ----
    print("\n  Test de Friedman (comparaison de modèles sur les folds) ")

    # On récupère les RMSE test par modèle sous forme de DataFrame (folds en ligne)
    df_friedman = pd.DataFrame(boxplot_values)

    # Test de Friedman
    from scipy.stats import friedmanchisquare

    friedman_stat, p_value = friedmanchisquare(*[df_friedman[col].dropna() for col in df_friedman.columns])

    print(f"Statistique de Friedman = {friedman_stat:.4f}")
    print(f"P-value = {p_value:.4f}")

    # ---- 11) Si significatif → test post-hoc Nemenyi ----
    if p_value < 0.05:
        print("\n→ Les performances diffèrent significativement (p < 0.05).")
        print("  Exécution du test post-hoc Nemenyi...\n")

        

        nemenyi_matrix = sp.posthoc_nemenyi_friedman(df_friedman)

        print("===== Matrice de Nemenyi (p-values) =====")
        print(nemenyi_matrix)

        # heatmap pour mieux visualiser
        plt.figure(figsize=(10, 8))
        sns.heatmap(nemenyi_matrix, annot=True, cmap="viridis")
        plt.title("Post-hoc Nemenyi — Comparaison pairwise des modèles")
        plt.show()

    else:
        print("\n→ Les performances ne diffèrent PAS significativement (p ≥ 0.05).")


    return results_df, best_model_name


In [ ]:
results_df, best = modelisation_regression_area1_visualisee(X_train, y_train, cv_splits=5)
results_df
print("Meilleur modèle :", best)

In [ ]:
results_df, best = modelisation_regression_area1_visualisee(X_train, y_train, cv_splits=5, log_activation=True)
print("Meilleur modèle :", best)
results_df

**Interprétation**

Les résultats montrent que Gradient Boosting est le modèle le plus performant pour prédire area1. Il obtient le plus faible RMSE sur le test, une bonne stabilité, et un temps d’exécution raisonnable, ce qui en fait le meilleur compromis précision-robustesse.

La régression linéaire arrive juste derrière : rapide et stable, mais un peu moins précise.
Les modèles comme Decision Tree, Random Forest, XGBoost, LightGBM, CatBoost ou KNN présentent soit un fort surapprentissage, soit une variabilité trop élevée, soit des erreurs nettement supérieures.

En résumé, Gradient Boosting est le meilleur choix global pour estimer la taille moyenne des cellules (area1).


## II. Optimisation des modèles


Pour optimiser la performance de nos modèles, nous allons utilisé plusieurs techniques à savoir:
* Le feature_engineering
* L'optimisation des hyperparamètres via RandomizedSearchCV


### 1. feature engineering

In [ ]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.cluster import KMeans

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import (
    RandomForestRegressor, GradientBoostingRegressor,
    BaggingRegressor, AdaBoostRegressor
)
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import scikit_posthocs as sp

def modelisation_regression_area1_visualisee(X, y, cv_splits=5, log_activation=False):
    """
    Compare plusieurs modèles de régression pour prédire 'area1' (RMSE).
    - Ajoute une colonne 'cluster' issue d'un KMeans(2)
    - Effectue une validation croisée manuelle (KFold)
    - Calcule RMSE train/test, std, temps d'entraînement/prédiction
    - Affiche 3 graphiques : RMSE train vs test (barres annotées), boxplot (variabilité),
      temps d'entraînement vs prédiction (barres annotées)
    - Calcule un score global (à maximiser) et renvoie le DataFrame des résultats et le meilleur modèle
    """

    # copie des données pour sécurité
    X = X.copy()
    
    if log_activation:
        y_log = np.log1p(y)
        y = y_log.copy().reset_index(drop=True)

    y = y.copy().reset_index(drop=True)
    X = X.reset_index(drop=True)

    # ---- 1) Ajouter cluster (KMeans 2) ----
    kmeans = KMeans(n_clusters=2, random_state=42)
    try:
        X["cluster"] = kmeans.fit_predict(X)
    except Exception:
        # si X contient des colonnes non numériques (peu probable ici), on essaie de sélectionner numériques
        numeric_cols = X.select_dtypes(include=np.number).columns.tolist()
        X["cluster"] = kmeans.fit_predict(X[numeric_cols])

    # ---- 2) Prétraitement (StandardScaler sur colonnes numériques) ----
    numeric_features = X.select_dtypes(include=np.number).columns.tolist()
    preprocessor = ColumnTransformer(
        [('num', StandardScaler(), numeric_features)],
        remainder='passthrough'
    )

    # ---- 3) Modèles de régression ----
    models = {
        "Linear Regression": LinearRegression(),
        "KNN": KNeighborsRegressor(n_neighbors=5),
        "Decision Tree": DecisionTreeRegressor(random_state=42),
        "Random Forest": RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1),
        "Gradient Boosting": GradientBoostingRegressor(n_estimators=200, learning_rate=0.1, random_state=42),
        "Bagging": BaggingRegressor(n_estimators=50, random_state=42),
        "AdaBoost": AdaBoostRegressor(n_estimators=200, learning_rate=0.1, random_state=42),
        "XGBoost": XGBRegressor(n_estimators=200, learning_rate=0.1, max_depth=5, random_state=42, verbosity=0),
        "LightGBM": LGBMRegressor(n_estimators=200, learning_rate=0.1, random_state=42),
        "CatBoost": CatBoostRegressor(iterations=200, learning_rate=0.1, depth=6, verbose=0, random_seed=42)
    }

    # ---- 4) Structures de stockage ----
    rmse_train_means = {}
    rmse_test_means = {}
    rmse_test_stds = {}
    train_times = {}
    pred_times = {}
    boxplot_values = {}

    kf = KFold(n_splits=cv_splits, shuffle=True, random_state=42)
    print(f"\n  Début de la validation croisée (CV={cv_splits})  \n")

    # ---- 5) Boucle sur modèles, CV manuelle ----
    for name, model in models.items():
        print(f"--- Modèle : {name} ---")
        pipeline = Pipeline([("preprocess", preprocessor), ("model", model)])

        rmse_train_folds = []
        rmse_test_folds = []
        times_train_folds = []
        times_pred_folds = []

        for fold_idx, (train_idx, test_idx) in enumerate(kf.split(X), 1):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

            # entraînement
            t0 = time.time()
            try:
                pipeline.fit(X_train, y_train)
            except Exception as e:
                print(f"  Erreur fit sur fold {fold_idx} pour {name} : {e}")
                # si échec, on ajoute NaN et continue
                rmse_train_folds.append(np.nan)
                rmse_test_folds.append(np.nan)
                times_train_folds.append(np.nan)
                times_pred_folds.append(np.nan)
                continue
            t1 = time.time()

            # prédiction
            t2 = time.time()
            y_pred_train = pipeline.predict(X_train)
            y_pred_test = pipeline.predict(X_test)
            t3 = time.time()

            # RMSE
            rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
            rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))

            rmse_train_folds.append(rmse_train)
            rmse_test_folds.append(rmse_test)
            times_train_folds.append(t1 - t0)
            times_pred_folds.append(t3 - t2)

            print(f"  Fold {fold_idx} — RMSE_train: {rmse_train:.4f}, RMSE_test: {rmse_test:.4f}, "
                  f"t_train: {t1-t0:.4f}s, t_pred: {t3-t2:.4f}s")

        # moyennes et std
        rmse_train_means[name] = np.nanmean(rmse_train_folds)
        rmse_test_means[name] = np.nanmean(rmse_test_folds)
        rmse_test_stds[name] = np.nanstd(rmse_test_folds)
        train_times[name] = np.nanmean(times_train_folds)
        pred_times[name] = np.nanmean(times_pred_folds)
        boxplot_values[name] = rmse_test_folds

        print(f"→ {name} Résumé : RMSE_train_mean = {rmse_train_means[name]:.4f}, "
              f"RMSE_test_mean = {rmse_test_means[name]:.4f}, RMSE_test_std = {rmse_test_stds[name]:.4f}\n")

    print("  Fin de la CV  \n")

    # ---- 6) DataFrame récapitulatif ----
    results_df = pd.DataFrame({
        "RMSE_train_mean": rmse_train_means,
        "RMSE_test_mean": rmse_test_means,
        "RMSE_test_std": rmse_test_stds,
        "Train_time": train_times,
        "Pred_time": pred_times
    })

    # ---- 7) Score global (à maximiser). On minimise RMSE, donc on prend négatif.
    alpha = 0.1   # pénalisation pour la variance
    beta = 0.01   # pénalisation pour le temps de prédiction
    results_df["Score_global"] = - (results_df["RMSE_test_mean"]
                                    + alpha * results_df["RMSE_test_std"]
                                    + beta * results_df["Pred_time"])

    best_model_name = results_df["Score_global"].idxmax()
    print("  Meilleur modèle identifié  ")
    print(results_df.loc[best_model_name])
    print("\n")

    # ---- 8) Fonctions d'annotation ----
    def annotate_bars(ax):
        for bar in ax.patches:
            h = bar.get_height()
            ax.annotate(f"{h:.3f}",
                        xy=(bar.get_x() + bar.get_width() / 2, h),
                        xytext=(0, 5),
                        textcoords="offset points",
                        ha="center", va="bottom", fontsize=9)
            

    

    # ---- 9) Graphiques ----
    models_list = list(rmse_train_means.keys())
    x = np.arange(len(models_list))
    width = 0.35

    # Graph 1 : RMSE train vs test
    fig, ax = plt.subplots(figsize=(12, 6))
    bars1 = ax.bar(x - width/2, [results_df.loc[m, "RMSE_train_mean"] for m in models_list], width, label="RMSE Train")
    bars2 = ax.bar(x + width/2, [results_df.loc[m, "RMSE_test_mean"] for m in models_list], width, label="RMSE Test")
    ax.set_xticks(x)
    ax.set_xticklabels(models_list, rotation=45, ha='right')
    ax.set_ylabel("RMSE")
    ax.set_title("RMSE moyen (Train vs Test)")
    ax.legend()
    annotate_bars(ax)
    plt.tight_layout()
    plt.show()

    # Graph 2 : Robustesse (boxplot RMSE test par fold)
    plt.figure(figsize=(12, 6))
    df_box = pd.DataFrame(boxplot_values)
    sns.boxplot(data=df_box)
    plt.title("Robustesse des modèles (variabilité RMSE-test sur les folds)")
    plt.ylabel("RMSE (test)")
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

    # Graph 3 : Temps entraînement vs prédiction
    fig, ax = plt.subplots(figsize=(12, 6))
    bars3 = ax.bar(x - width/2, [results_df.loc[m, "Train_time"] for m in models_list], width, label="Train time (s)")
    bars4 = ax.bar(x + width/2, [results_df.loc[m, "Pred_time"] for m in models_list], width, label="Pred time (s)")
    ax.set_xticks(x)
    ax.set_xticklabels(models_list, rotation=45, ha='right')
    ax.set_title("Temps moyen d'entraînement et de prédiction")
    ax.legend()
    annotate_bars(ax)
    plt.tight_layout()
    plt.show()


    # ---- 10) Test de Friedman ----
    print("\n  Test de Friedman (comparaison de modèles sur les folds) ")

    # On récupère les RMSE test par modèle sous forme de DataFrame (folds en ligne)
    df_friedman = pd.DataFrame(boxplot_values)

    # Test de Friedman
    from scipy.stats import friedmanchisquare

    friedman_stat, p_value = friedmanchisquare(*[df_friedman[col].dropna() for col in df_friedman.columns])

    print(f"Statistique de Friedman = {friedman_stat:.4f}")
    print(f"P-value = {p_value:.4f}")

    # ---- 11) Si significatif → test post-hoc Nemenyi ----
    if p_value < 0.05:
        print("\n→ Les performances diffèrent significativement (p < 0.05).")
        print("  Exécution du test post-hoc Nemenyi...\n")

        

        nemenyi_matrix = sp.posthoc_nemenyi_friedman(df_friedman)

        print("===== Matrice de Nemenyi (p-values) =====")
        print(nemenyi_matrix)

        # heatmap pour mieux visualiser
        plt.figure(figsize=(10, 8))
        sns.heatmap(nemenyi_matrix, annot=True, cmap="viridis")
        plt.title("Post-hoc Nemenyi — Comparaison pairwise des modèles")
        plt.show()

    else:
        print("\n→ Les performances ne diffèrent PAS significativement (p ≥ 0.05).")


    return results_df, best_model_name


In [ ]:
X_f_eng, y_f_eng=add_regression_features_area1(train)
results_df, best = modelisation_regression_area1_visualisee(X_f_eng, y_train, cv_splits=5, log_activation=True)
print("Meilleur modèle :", best)
results_df


**Interprétation**

Le Gradient Boosting est le meilleur modèle : plus faible RMSE test (0.0218), stabilité correcte (std 0.0052) et temps de prédiction très faible (0.009 s).
XGBoost suit de près (RMSE 0.0238) mais est un peu moins stable et plus lent.
Linear Regression et Random Forest offrent des performances acceptables (RMSE ≈ 0.028) et des temps très rapides pour la prédiction.
Le Decision Tree apprend trop parfaitement (RMSE train = 0) mais généralise mal (test 0.037), signe d’overfitting.
Les moins bons modèles sont LightGBM, CatBoost et AdaBoost (RMSE > 0.038), avec une variabilité plus forte et des temps d’entraînement élevés.

A partir du test de Friedman on peut dire : 

Les modèles Linear Regression, Decision Tree, Random Forest, Bagging, AdaBoost, Gradient Boosting et XGBoost sont tous statistiquement équivalents, offrant les meilleures performances.
KNN est significativement moins performant.
LightGBM et CatBoost ne sont pas significativement meilleurs que KNN et restent derrière les meilleurs modèles.

## 3. Optimisation des hyperparamètres via RandomizedSearchCV

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, mean_squared_error

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import (
    RandomForestRegressor, GradientBoostingRegressor,
    BaggingRegressor, AdaBoostRegressor
)
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

import numpy as np


# --- Scorer RMSE (Sklearn demande un score → on met négatif) ---
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

rmse_scorer = make_scorer(rmse, greater_is_better=False)


def hyperparam_optimization_regression(X, y, cv_splits=5, n_iter=40, random_state=42):
    """
    Optimisation des hyperparamètres pour plusieurs modèles de régression
    (cible : area1) avec RandomizedSearchCV utilisant le RMSE.
    """

    numeric_features = X.select_dtypes(include=np.number).columns.tolist()

    preprocessor = ColumnTransformer(
        [('num', StandardScaler(), numeric_features)],
        remainder='passthrough'
    )

    # --- Modèles et grilles d’hyperparamètres ---
    model_params = {

        "Ridge": {
            "model": Ridge(),
            "params": {
                "model__alpha": [0.01, 0.1, 1, 10, 50, 100]
            }
        },

        "Lasso": {
            "model": Lasso(),
            "params": {
                "model__alpha": [0.0001, 0.001, 0.01, 0.1, 1]
            }
        },

        "Decision Tree": {
            "model": DecisionTreeRegressor(random_state=random_state),
            "params": {
                "model__max_depth": [None, 3, 5, 7, 10],
                "model__min_samples_split": [2, 5, 10],
                "model__min_samples_leaf": [1, 2, 5]
            }
        },

        "Random Forest": {
            "model": RandomForestRegressor(random_state=random_state),
            "params": {
                "model__n_estimators": [200, 400, 600],
                "model__max_depth": [None, 5, 10],
                "model__min_samples_split": [2, 5, 10]
            }
        },

        "Gradient Boosting": {
            "model": GradientBoostingRegressor(random_state=random_state),
            "params": {
                "model__n_estimators": [100, 200, 400],
                "model__learning_rate": [0.01, 0.05, 0.1],
                "model__max_depth": [2, 3, 4]
            }
        },

        "Bagging": {
            "model": BaggingRegressor(random_state=random_state),
            "params": {
                "model__n_estimators": [50, 100, 200],
                "model__max_samples": [0.5, 0.7, 1.0]
            }
        },

        "AdaBoost": {
            "model": AdaBoostRegressor(random_state=random_state),
            "params": {
                "model__n_estimators": [100, 200, 400],
                "model__learning_rate": [0.01, 0.1, 0.5, 1.0]
            }
        },

        "XGBoost": {
            "model": XGBRegressor(random_state=random_state),
            "params": {
                "model__n_estimators": [200, 400, 600],
                "model__learning_rate": [0.01, 0.05, 0.1],
                "model__max_depth": [3, 5, 7]
            }
        },

        "LightGBM": {
            "model": LGBMRegressor(random_state=random_state),
            "params": {
                "model__n_estimators": [200, 400, 600],
                "model__learning_rate": [0.01, 0.05, 0.1],
                "model__num_leaves": [20, 31, 50]
            }
        },

        "CatBoost": {
            "model": CatBoostRegressor(verbose=0, random_seed=random_state),
            "params": {
                "model__depth": [4, 6, 8],
                "model__iterations": [200, 400, 600],
                "model__learning_rate": [0.01, 0.05, 0.1]
            }
        }
    }

    best_models = {}
    best_params = {}

    # --- Boucle d’optimisation ---
    for name, mp in model_params.items():
        print(f"\n  Optimisation de {name}  ")

        pipeline = Pipeline([
            ('preprocess', preprocessor),
            ('model', mp["model"])
        ])

        search = RandomizedSearchCV(
            estimator=pipeline,
            param_distributions=mp["params"],
            scoring=rmse_scorer,
            n_iter=n_iter,
            cv=cv_splits,
            verbose=1,
            random_state=random_state,
            n_jobs=-1
        )

        search.fit(X, y)

        best_models[name] = search.best_estimator_
        best_params[name] = search.best_params_

        print("→ Best RMSE :", round(-search.best_score_, 4))
        print("→ Best params :", search.best_params_)

    return best_models, best_params


In [ ]:
best_models, best_params = hyperparam_optimization_regression(X_train, y_train)

In [ ]:
best_params

## Evaluation sur les données de test

In [ ]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.cluster import KMeans

from sklearn.linear_model import Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, BaggingRegressor, AdaBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

def modele_regression_train_test(X_train, y_train, X_test, y_test, log_activation=False):
    """
    Entraîne plusieurs modèles sur X_train/y_train, prédit sur X_test/y_test.
    - Ajoute une colonne 'cluster' issue d'un KMeans(2)
    - Standardisation des colonnes numériques
    - Affiche les métriques R2, MAE, RMSE sous forme de graphiques à barres
    - Affiche les temps d'entraînement et de prédiction
    """
    # Copie des données pour sécurité
    X_train = X_train.copy()
    X_test = X_test.copy()
    y_train = y_train.copy()
    y_test = y_test.copy()
    
    if log_activation:
        y_train = np.log1p(y_train)
        y_test = np.log1p(y_test)

    # ---- 1) Ajouter cluster (KMeans 2) ----
    kmeans = KMeans(n_clusters=2, random_state=42)
    try:
        X_train["cluster"] = kmeans.fit_predict(X_train)
        X_test["cluster"] = kmeans.predict(X_test)
    except Exception:
        numeric_cols = X_train.select_dtypes(include=np.number).columns.tolist()
        X_train["cluster"] = kmeans.fit_predict(X_train[numeric_cols])
        X_test["cluster"] = kmeans.predict(X_test[numeric_cols])

    # ---- 2) Prétraitement (StandardScaler sur colonnes numériques) ----
    numeric_features = X_train.select_dtypes(include=np.number).columns.tolist()
    preprocessor = ColumnTransformer(
        [('num', StandardScaler(), numeric_features)],
        remainder='passthrough'
    )

    # ---- 3) Modèles avec hyperparamètres ----
    models = {
        "Ridge": Ridge(alpha=0.1),
        "Lasso": Lasso(alpha=0.1),
        "Decision Tree": DecisionTreeRegressor(min_samples_split=2, min_samples_leaf=1, max_depth=7, random_state=42),
        "Random Forest": RandomForestRegressor(n_estimators=400, min_samples_split=2, max_depth=None, random_state=42, n_jobs=-1),
        "Gradient Boosting": GradientBoostingRegressor(n_estimators=200, max_depth=2, learning_rate=0.1, random_state=42),
        "Bagging": BaggingRegressor(n_estimators=200, max_samples=1.0, random_state=42),
        "AdaBoost": AdaBoostRegressor(n_estimators=200, learning_rate=1.0, random_state=42),
        "XGBoost": XGBRegressor(n_estimators=600, max_depth=3, learning_rate=0.05, random_state=42, verbosity=0),
        "LightGBM": LGBMRegressor(num_leaves=20, n_estimators=200, learning_rate=0.05, random_state=42),
        "CatBoost": CatBoostRegressor(iterations=600, learning_rate=0.1, depth=4, verbose=0, random_seed=42)
    }

    # ---- 4) Stockage métriques ----
    results = {
        "R2": {},
        "MAE": {},
        "RMSE": {},
        "Train_time": {},
        "Pred_time": {}
    }

    # ---- 5) Boucle sur modèles ----
    for name, model in models.items():
        pipeline = Pipeline([("preprocess", preprocessor), ("model", model)])
        print(f"--- Entraînement : {name} ---")

        # Entraînement
        t0 = time.time()
        pipeline.fit(X_train, y_train)
        t1 = time.time()

        # Prédiction
        t2 = time.time()
        y_pred = pipeline.predict(X_test)
        t3 = time.time()

        # Calcul métriques
        results["R2"][name] = r2_score(y_test, y_pred)
        results["MAE"][name] = mean_absolute_error(y_test, y_pred)
        results["RMSE"][name] = np.sqrt(mean_squared_error(y_test, y_pred))
        results["Train_time"][name] = t1 - t0
        results["Pred_time"][name] = t3 - t2

        print(f"R2={results['R2'][name]:.4f}, MAE={results['MAE'][name]:.4f}, RMSE={results['RMSE'][name]:.4f}, "
              f"t_train={results['Train_time'][name]:.3f}s, t_pred={results['Pred_time'][name]:.3f}s")

    # ---- 6) DataFrame récapitulatif ----
    results_df = pd.DataFrame(results)

    # ---- 7) Graphiques ----
    def plot_metric(metric, title):
        plt.figure(figsize=(12,6))
        sns.barplot(x=results_df.index, y=results_df[metric], palette="viridis")
        plt.title(title)
        plt.ylabel(metric)
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()

    plot_metric("R2", "R² des modèles sur X_test")
    plot_metric("MAE", "MAE des modèles sur X_test")
    plot_metric("RMSE", "RMSE des modèles sur X_test")

    # Graphique temps
    plt.figure(figsize=(12,6))
    x = np.arange(len(results_df))
    width = 0.35
    plt.bar(x - width/2, results_df["Train_time"], width, label="Train time (s)")
    plt.bar(x + width/2, results_df["Pred_time"], width, label="Pred time (s)")
    plt.xticks(x, results_df.index, rotation=45, ha='right')
    plt.ylabel("Temps (s)")
    plt.title("Temps d'entraînement et de prédiction")
    plt.legend()
    plt.tight_layout()
    plt.show()

    return results_df


**Interprétation**


Random Forest : meilleur compromis précision/robustesse. R² très élevé (0.9981), MAE et RMSE les plus bas, temps d’entraînement/prédiction corrects.

Bagging : R² proche de Random Forest (0.9979), MAE/RMSE légèrement plus élevés, mais temps d’entraînement plus long (2.7 s).

XGBoost : R² = 0.9967, MAE/RMSE un peu plus hauts que Random Forest, temps d’entraînement 1.6 s, rapide en prédiction.

Gradient Boosting : R² correct (0.9964), MAE/RMSE un peu plus élevés, temps d’entraînement 1 s, prédiction très rapide.

Decision Tree, Ridge, Lasso : très rapides à entraîner, R² déjà très bon (>0.995), mais erreurs légèrement supérieures à Random Forest/Bagging/XGBoost.

AdaBoost, CatBoost, LightGBM : R² et erreurs moins bonnes, à éviter pour ce dataset malgré certains temps d’entraînement raisonnables.



Conclusion : En trouvant les meilleurs hyperparamètres , le Random Forest devient  le meilleur modèle pour la tache , suivi de près par Bagging et XGBoost pour ceux qui acceptent un léger compromis sur les performances, le Gradient Boosting demeure une meilleure option.

Nous allons faire toutes ces étapes dans une seule fonction

Elle prend en entrée notre dataframe et nous retourne les différentes parties ou données dont nous aurons besoins pour l'entrainnement, le test, et l'interprétabilité.

# Partie 5  Discussions finales

**Synthèse de l’étude sur la base AMA (UCI)**

Cette étude a pour objectif de prédire deux aspects essentiels à partir de données tabulaires extraites d’images PAAF :

Le diagnostic (benin/malin)

La valeur de l’aire area1 des noyaux cellulaires.

Elle explore différents modèles classiques de machine learning pour identifier ceux qui offrent le meilleur compromis entre précision, robustesse et rapidité, sans recourir à des modèles complexes type CNN, car les caractéristiques pertinentes ont été pré-extraites.

**Partie 1 : Prédiction du diagnostic**



Objectif : Identifier correctement les masses mammaires bénignes et malignes.

Meilleur modèle : Régression Logistique

 F1-test = 963855,  F1-train = 0.967552 donc pas d’overfitting.

Très rapide : entraînement 0.03 s, prédiction 0.01 s.

Capacité élevée à détecter les cas positifs (malins).

Modèles complexes (Random Forest, Gradient Boosting, XGBoost, LightGBM)

Ajustement parfait sur le train (1.00), mais moins performant sur test (≈0.92–0.94).

Sont sensibles au bruit et au surapprentissage.

Conclusion opérationnelle : La Régression Logistique est le modèle le plus sûr et stable pour un usage réel, combinant rapidité et détection fiable des cas positifs.

**Partie 2 : Prédiction de area1**



Objectif : Prédire précisément l’aire des noyaux cellulaires (area1).

Meilleur modèle : Random Forest

R² = 0.9981, MAE = 7.55, RMSE = 14.90 → meilleure précision et robustesse.

Temps d’entraînement/prédiction corrects.

Autres modèles performants :

Bagging : R² = 0.9979, MAE/RMSE légèrement supérieurs, plus lent à entraîner.

XGBoost : R² = 0.9967, MAE/RMSE un peu plus élevés, rapide en prédiction.

Modèles à éviter : LightGBM, AdaBoost, CatBoost → erreurs plus grandes malgré certains temps d’entraînement corrects.

Conclusion : Random Forest offre le meilleur compromis pour la prédiction numérique de area1.




**Combinaison des deux parties : bénéfices**

Prédiction diagnostique + prédiction d’aire : combiner les deux approches permet :

D’avoir un diagnostic fiable et une quantification précise des caractéristiques tumorales.

De prioriser les patients à risque élevé en fonction de la taille/aire des noyaux et du diagnostic.

D’orchestrer les modèles :

Régression Logistique pour le diagnostic rapide et stable.

Random Forest (ou Bagging/XGBoost selon contraintes de temps) pour prédiction précise des valeurs continues.

Cette approche hybride maximise la robustesse et l’efficacité opérationnelle.



**Conclusion générale**

L’étude montre que des modèles classiques bien paramétrés peuvent surpasser les modèles complexes sur ce type de données tabulaires, extraites d’images médicales.

Régression Logistique → meilleure pour le diagnostic, rapide et stable.

Random Forest → meilleure pour la prédiction de area1, précise et robuste.

La combinaison des deux modèles permet d’avoir à la fois précision diagnostique et quantification des caractéristiques tumorales, ce qui est très utile pour la prise de décision clinique et la priorisation des cas.

Cette approche illustre que l’orchestration de modèles spécialisés selon la nature de la tâche (classification vs régression) est une stratégie efficace pour tirer le meilleur parti des données.

## **Conclusion  du projet**

Ce projet nous a permis de mettre en pratique les concepts liés aux réseaux de neurones, et plus particulièrement à l’utilisation d’un autoencodeur. Il a constitué une véritable opportunité pour nous initier à la conception autonome de modèles neuronaux, en développant des compétences essentielles telles que l’anticipation des problèmes, la gestion des erreurs et l’amélioration de nos méthodes de recherche grâce à l’exploration de diverses sources et documentations techniques.

Nous tenons à exprimer notre gratitude à notre enseignant pour cette opportunité, ainsi que pour le temps et l’énergie précieux consacrés à la préparation et à l’encadrement de ce projet.
